In [1]:
import os
import time
import asyncio
import numpy as np

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:

from libs.quotation import Quotation
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

ModuleNotFoundError: No module named 'libs.cython.compute'

In [4]:
policy = asyncio.WindowsSelectorEventLoopPolicy()
asyncio.set_event_loop_policy(policy)

In [16]:
class QuotationMonitor(DailyData):
    
    def __init__(self, symbols, check_points, buffers=None):
        super().__init__(symbols, check_points, buffers)
        self.q = Quotation(symbols)
        
        
    async def snapshot(self, idx, delay=0):
        await asyncio.sleep(max(delay,0))
        await self.q.snapshot(array=self.snapshots[idx])
        
        # calculate statistic
        time_lapse = self.get_time_lapse(idx)
        ma5pm_anchor_idx = self.get_ma5pm_anchor_idx(idx)
        fs5p = self.snapshots[ma5pm_anchor_idx]
        compute_stats(self.snapshots[idx], self.basics, self.statistic[idx], fs5p, time_lapse)
        
        
    async def start(self):
        for _, check_point in enumerate(self.check_points):
            
            if time.time() > check_point:
                continue
                
            await self.snapshot(_, delay=(check_point-time.time()))
            

In [6]:
from libs.tdx import TDX
from libs.utils import Utils

In [ ]:
Utils.update_symbols()

In [7]:
check_points = Utils.get_check_points()
symbols = Utils.get_running_symbols()

In [17]:
qm = QuotationMonitor(symbols, check_points)

In [18]:
await qm.prepare(symbols, check_points)

In [20]:
await qm.start()

In [21]:
qm.save()

文件 [ D:\workspace\python\Securities\storage\20210125.hdf5 ] 已经存在，将被删除 ... ... 已被删除
行情 已经 写入文件： D:\workspace\python\Securities\storage\20210125.hdf5


In [28]:
qm.statistic[-1000,:,:]

array([[0.05, 21.88, 0.74, ..., 20.49, 19.80, 19.21],
       [-0.65, 28.87, 0.91, ..., 29.37, 29.03, 29.43],
       [-1.88, 19.29, 0.81, ..., 19.70, 20.71, 24.33],
       ...,
       [-2.13, 56.52, 0.63, ..., nan, nan, nan],
       [1.33, 57.57, 0.89, ..., 59.40, 58.16, 59.86],
       [4.94, 105.03, 1.11, ..., 84.49, 80.71, 75.03]], dtype=float32)

In [ ]:

time.strftime('%H%M%S', time.localtime(1610673300))